<a href="https://colab.research.google.com/github/manoj-mathivanan/pythonML/blob/main/share_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# prompt: create an algorithm which  Identifies multiple buying and selling points in a time series of stock prices with no window limit and output the profit and rank it according to the profit. Also include option where I sell first and then buy later.

import pandas as pd

def find_optimal_trades(prices):
    """
    Identifies multiple buying and selling points in a time series of stock prices.

    Args:
        prices: A list or pandas Series of stock prices.

    Returns:
        A pandas DataFrame with columns 'Buy', 'Sell', 'Profit', and 'Rank',
        sorted by profit in descending order.  Includes trades where a sell occurs before a buy.
    """

    n = len(prices)
    trades = []

    for i in range(n):
        for j in range(i + 1, n):
            profit = prices[j] - prices[i]
            trades.append({'Buy': prices[i], 'Sell': prices[j], 'Profit': profit})


    # Allow selling before buying (as requested)
    for i in range(n):
      for j in range(i+1,n):
        profit = prices[i] - prices[j]
        trades.append({'Buy': prices[j], 'Sell': prices[i], 'Profit':profit})


    df = pd.DataFrame(trades)
    df['Rank'] = df['Profit'].rank(ascending=False, method='dense')
    df = df.sort_values('Profit', ascending=False)

    return df

# Example Usage:
prices = [10, 12, 8, 15, 7, 18, 11, 20, 9]
optimal_trades = find_optimal_trades(prices)
print(optimal_trades)

# Example with Pandas Series:
price_series = pd.Series([10, 12, 8, 15, 7, 18, 11, 20, 9])
optimal_trades_series = find_optimal_trades(price_series)
print("\nUsing Pandas Series:")
optimal_trades_series


    Buy  Sell  Profit  Rank
28    7    20      13   1.0
19    8    20      12   2.0
26    7    18      11   3.0
71    9    20      11   3.0
17    8    18      10   4.0
..  ...   ...     ...   ...
42   20    10     -10  23.0
62   18     7     -11  24.0
35   20     9     -11  24.0
55   20     8     -12  25.0
64   20     7     -13  26.0

[72 rows x 4 columns]

Using Pandas Series:


,Buy,Sell,Profit,Rank
28,7,20,13,1.0
19,8,20,12,2.0
26,7,18,11,3.0
71,9,20,11,3.0
17,8,18,10,4.0
...,...,...,...,...
42,20,10,-10,23.0
62,18,7,-11,24.0
35,20,9,-11,24.0
55,20,8,-12,25.0
